In [35]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
pd.set_option("display.max_columns",None)
# Load your CSV data into a pandas DataFrame
df = pd.read_csv('cleaned_data.csv')
df

,event_description,location_description,location_accuracy,landslide_trigger,landslide_size,landslide_setting,landslide_category,fatality_count,injury_count,storm_name,photo_link,notes,event_import_source,event_import_id,country_name,country_code,admin_division_population,rain,earthquake,downpour,snow,other,small_scale,medium_scale,large_scale,home_destroyed,mudslide,landslide
0,(CBS/AP) At least 10 people died and as many a...,"San Ramon district, 195 miles northeast of the...",10km,downpour,large,unknown,landslide,10.0,0.0,NaN,NaN,NaN,glc,973.0,Peru,PE,14708.0,1,0,1,0,0,0,0,1,0,0,1
1,"One person was killed in Dailekh district, pol...",Dailekh district,unknown,monsoon,medium,unknown,landslide,1.0,0.0,NaN,NaN,NaN,glc,1067.0,Nepal,NP,20908.0,1,0,0,0,0,0,1,0,0,0,1
2,"3 killed, including 2 children when rocks fell...","calle Granjas, Ampliación Miguel de la Madrid,...",10km,tropical_cyclone,medium,unknown,complex,3.0,0.0,Tropical Storm Henrietta,http://www.cronica.com.mx/oimagenes11/1/f94ae8...,NaN,glc,225.0,Mexico,MX,3191.0,0,0,0,0,1,0,1,0,1,0,0
3,Yunnan has so far reported 40 deaths and 43 pe...,"Kunming, Yunnan",25km,downpour,medium,unknown,complex,4.0,0.0,NaN,NaN,NaN,glc,874.0,China,CN,1023674.0,1,0,1,0,0,0,1,0,0,0,0
4,A landslide hit the village in barangay Gadgar...,"Gadgaron village of Matnog town, Sorsogon",5km,downpour,medium,unknown,landslide,0.0,1.0,NaN,NaN,NaN,glc,955.0,Philippines,PH,6721.0,1,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,"With the recovery of three more bodies, the to...","Darjeeling Sadar, Darjeeling West Bengal",25km,tropical_cyclone,medium,unknown,landslide,14.0,0.0,Cyclone Aila,NaN,NaN,glc,1030.0,India,IN,123797.0,0,0,0,0,1,0,1,0,1,0,1
1508,Clearing operations are ongoing in many roads...,"Kennon Road, Benguet",25km,tropical_cyclone,medium,unknown,landslide,1.0,0.0,Utor,NaN,NaN,glc,5314.0,Philippines,PH,2653.0,0,0,0,0,1,0,1,0,0,0,1
1509,Specific information was not given. Multiple l...,NaN,unknown,monsoon,unknown,unknown,unknown,40.0,20.0,NaN,NaN,NaN,glc,9261.0,Nepal,NP,21867.0,1,0,0,0,0,0,0,0,0,0,0
1510,SÃO PAULO — Brazilian authorities say a couple...,"Maua, Sao Paulo",5km,downpour,medium,unknown,mudslide,2.0,0.0,NaN,NaN,NaN,glc,2935.0,Brazil,BR,386069.0,1,0,1,0,0,0,1,0,1,1,0


In [24]:
df.columns

Index(['event_description', 'location_description', 'location_accuracy',
       'landslide_trigger', 'landslide_size', 'landslide_setting',
       'landslide_category', 'fatality_count', 'injury_count', 'storm_name',
       'photo_link', 'notes', 'event_import_source', 'event_import_id',
       'country_name', 'country_code', 'admin_division_population', 'rain',
       'earthquake', 'downpour', 'snow', 'other', 'small_scale',
       'medium_scale', 'large_scale', 'home_destroyed', 'mudslide',
       'landslide'],
      dtype='object')

In [64]:
df_new=df[['location_accuracy', 'fatality_count', 'injury_count', 'admin_division_population', 'rain',
       'earthquake', 'downpour', 'snow', 'other', 'small_scale',
       'medium_scale', 'large_scale', 'home_destroyed', 'mudslide',
       'landslide']]
df_new

,location_accuracy,fatality_count,injury_count,admin_division_population,rain,earthquake,downpour,snow,other,small_scale,medium_scale,large_scale,home_destroyed,mudslide,landslide
0,10km,10.0,0.0,14708.0,1,0,1,0,0,0,0,1,0,0,1
1,unknown,1.0,0.0,20908.0,1,0,0,0,0,0,1,0,0,0,1
2,10km,3.0,0.0,3191.0,0,0,0,0,1,0,1,0,1,0,0
3,25km,4.0,0.0,1023674.0,1,0,1,0,0,0,1,0,0,0,0
4,5km,0.0,1.0,6721.0,1,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,25km,14.0,0.0,123797.0,0,0,0,0,1,0,1,0,1,0,1
1508,25km,1.0,0.0,2653.0,0,0,0,0,1,0,1,0,0,0,1
1509,unknown,40.0,20.0,21867.0,1,0,0,0,0,0,0,0,0,0,0
1510,5km,2.0,0.0,386069.0,1,0,1,0,0,0,1,0,1,1,0


In [68]:
#getting rid of km in landslide accuracy
df_new['location_accuracy']=df_new['location_accuracy'].apply(lambda x: x.replace("km",""))
df_new['location_accuracy']=df_new['location_accuracy'].apply(lambda x: x.replace('unknown','0'))
df_new['location_accuracy']=df_new['location_accuracy'].apply(lambda x: x.replace('exact','0'))
df_new['fatality_count']=df_new['fatality_count'].apply(lambda x: int(x))
df_new['injury_count']=df_new['injury_count'].apply(lambda x: int(x))
df_new['admin_divison_population']=df_new['admin_division_population'].apply(lambda x: int(x))
df_new

C:\Users\ebeng\AppData\Local\Temp\ipykernel_9996\246950906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['location_accuracy']=df_new['location_accuracy'].apply(lambda x: x.replace("km",""))
C:\Users\ebeng\AppData\Local\Temp\ipykernel_9996\246950906.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['location_accuracy']=df_new['location_accuracy'].apply(lambda x: x.replace('unknown','0'))
C:\Users\ebeng\AppData\Local\Temp\ipykernel_9996\246950906.py:4: SettingWithCopyWarning: 
A value is

,location_accuracy,fatality_count,injury_count,admin_division_population,rain,earthquake,downpour,snow,other,small_scale,medium_scale,large_scale,home_destroyed,mudslide,landslide,admin_divison_population
0,10,10,0,14708.0,1,0,1,0,0,0,0,1,0,0,1,14708
1,0,1,0,20908.0,1,0,0,0,0,0,1,0,0,0,1,20908
2,10,3,0,3191.0,0,0,0,0,1,0,1,0,1,0,0,3191
3,25,4,0,1023674.0,1,0,1,0,0,0,1,0,0,0,0,1023674
4,5,0,1,6721.0,1,0,1,0,0,0,1,0,0,0,1,6721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,25,14,0,123797.0,0,0,0,0,1,0,1,0,1,0,1,123797
1508,25,1,0,2653.0,0,0,0,0,1,0,1,0,0,0,1,2653
1509,0,40,20,21867.0,1,0,0,0,0,0,0,0,0,0,0,21867
1510,5,2,0,386069.0,1,0,1,0,0,0,1,0,1,1,0,386069


In [200]:
df_n1.columns

Index(['location_accuracy', 'fatality_count', 'injury_count',
       'admin_division_population', 'rain', 'small_scale', 'medium_scale',
       'large_scale', 'home_destroyed', 'mudslide', 'landslide', '1k_10k',
       '10k_100k', '100k_1m', '1m_10m'],
      dtype='object')

In [199]:
df_n1['admin_division_population']
pd.set_option("display.max_rows",None)
len(df_n1[df_n1['admin_division_population'].apply(lambda x: x<1000)])
df_n1['1k_10k']=df_n1['admin_division_population'].apply(lambda x: 1 if (x>1000 and x<10000) else 0)
df_n1['10k_100k']=df_n1['admin_division_population'].apply(lambda x: 1 if (x>10000 and x<100000) else 0)
df_n1['100k_1m']=df_n1['admin_division_population'].apply(lambda x: 1 if (x>100000 and x<1000000) else 0)
df_n1['1m_10m']=df_n1['admin_division_population'].apply(lambda x: 1 if (x>1000000 and x<10000000) else 0) 
df_n1

C:\Users\ebeng\AppData\Local\Temp\ipykernel_9996\4066649538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n1['1k_10k']=df_n1['admin_division_population'].apply(lambda x: 1 if (x>1000 and x<10000) else 0)
C:\Users\ebeng\AppData\Local\Temp\ipykernel_9996\4066649538.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n1['10k_100k']=df_n1['admin_division_population'].apply(lambda x: 1 if (x>10000 and x<100000) else 0)
C:\Users\ebeng\AppData\Local\Temp\ipykernel_9996\4066649538.py:6: SettingWithCopyW

,location_accuracy,fatality_count,injury_count,admin_division_population,rain,small_scale,medium_scale,large_scale,home_destroyed,mudslide,landslide,1k_10k,10k_100k,100k_1m,1m_10m
0,10,10,0,14708.0,1,0,0,1,0,0,1,0,1,0,0
1,0,1,0,20908.0,1,0,1,0,0,0,1,0,1,0,0
2,10,3,0,3191.0,0,0,1,0,1,0,0,1,0,0,0
3,25,4,0,1023674.0,1,0,1,0,0,0,0,0,0,0,1
4,5,0,1,6721.0,1,0,1,0,0,0,1,1,0,0,0
5,5,7,0,802639.0,1,0,1,0,1,0,1,0,0,1,0
6,5,2,0,64409.0,1,0,1,0,1,0,1,0,1,0,0
7,50,2,0,12294193.0,1,0,1,0,0,0,1,0,0,0,0
8,10,2,0,4164.0,1,0,1,0,0,0,1,1,0,0,0
9,5,6,0,116925.0,1,0,1,0,1,0,1,0,0,1,0


In [201]:
df_new=df_n1[['fatality_count', 'injury_count', 'location_accuracy','rain', 'small_scale', 'medium_scale',
       'large_scale', 'home_destroyed', 'mudslide', 'landslide', '1k_10k',
       '10k_100k', '100k_1m', '1m_10m']]

In [202]:
df_new

,fatality_count,injury_count,location_accuracy,rain,small_scale,medium_scale,large_scale,home_destroyed,mudslide,landslide,1k_10k,10k_100k,100k_1m,1m_10m
0,10,0,10,1,0,0,1,0,0,1,0,1,0,0
1,1,0,0,1,0,1,0,0,0,1,0,1,0,0
2,3,0,10,0,0,1,0,1,0,0,1,0,0,0
3,4,0,25,1,0,1,0,0,0,0,0,0,0,1
4,0,1,5,1,0,1,0,0,0,1,1,0,0,0
5,7,0,5,1,0,1,0,1,0,1,0,0,1,0
6,2,0,5,1,0,1,0,1,0,1,0,1,0,0
7,2,0,50,1,0,1,0,0,0,1,0,0,0,0
8,2,0,10,1,0,1,0,0,0,1,1,0,0,0
9,6,0,5,1,0,1,0,1,0,1,0,0,1,0


In [327]:
import numpy as np
from sklearn.model_selection import train_test_split
np_arr = df_new.values
np_arr = np_arr.astype('int32') #convert to int
X=np_arr[:,[2,3,7,8,9,10,11,12,13]]
y=np_arr[:,[0,1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [328]:
import tensorflow as tf
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(9,), name='input_layer'),
    keras.layers.Dense(32, activation='relu', name='hidden_layer_1'),])
model.add(layers.Dense(2, activation='softmax'))
optimizer=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer,  loss='categorical_crossentropy', metrics=['accuracy'])

In [329]:
res = model.fit(X_train, y_train, epochs=5, batch_size=50, validation_data=(X_test, y_test))
res

Epoch 1/5
23/23 [==============================] - 1s 11ms/step - loss: 6.6377 - accuracy: 0.8748 - val_loss: 5.0251 - val_accuracy: 0.8598
Epoch 2/5
23/23 [==============================] - 0s 5ms/step - loss: 6.8453 - accuracy: 0.8792 - val_loss: 5.2826 - val_accuracy: 0.8598
Epoch 3/5
23/23 [==============================] - 0s 6ms/step - loss: 7.1216 - accuracy: 0.8827 - val_loss: 5.6483 - val_accuracy: 0.8624
Epoch 4/5
23/23 [==============================] - 0s 6ms/step - loss: 7.6123 - accuracy: 0.8836 - val_loss: 6.2011 - val_accuracy: 0.8677
Epoch 5/5
23/23 [==============================] - 0s 4ms/step - loss: 8.0033 - accuracy: 0.8862 - val_loss: 6.7430 - val_accuracy: 0.8677


In [344]:
score=model.evaluate(X_test,y_test,verbose=0)
score

[6.74296236038208, 0.8677248954772949]

In [351]:
deets=[10,1,1,1,0,0,1,0,0]
X_new=np.array([deets])
predictions = model.predict(X_new)
predictions[[0]]*=10
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
result = (np.ceil(predictions)).astype(int)
print("Predictions:", result)     # Print the predictions(results) as int

1/1 [==============================] - 0s 37ms/step
Predictions: [[8 3]]
[[7.928757 2.071243]]
